In [2]:
import cv2
import sys
sys.path.insert(0, '/home/caron/Bureau/Model/HexagonalScale/ST-GNN-for-wildifre-prediction/Prediction')

from tools import *
from config import *

def calculate_iou(image1, image2):
    
    # Vérifier que les deux images sont de la même taille
    if image1.shape != image2.shape:
        raise ValueError("Les deux images doivent être de la même taille pour calculer l'IoU.")
    
    # Binariser les images (seuiling pour les valeurs de pixels 0 ou 255)
    _, binary_image1 = cv2.threshold(image1, 127, 255, cv2.THRESH_BINARY)
    _, binary_image2 = cv2.threshold(image2, 127, 255, cv2.THRESH_BINARY)
    
    # Calculer l'intersection et l'union
    intersection = np.logical_and(binary_image1, binary_image2).sum()
    union = np.logical_or(binary_image1, binary_image2).sum()
    
    # Calculer l'IoU
    iou = intersection / union if union != 0 else 0
    return iou

/home/caron/Bureau/.conda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Nom de l'hôte : caron-Precision-7780
Adresse IP locale : 127.0.1.1


/home/caron/Bureau/.conda/lib/python3.9/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Nom de l'hôte : caron-Precision-7780
Adresse IP locale : 127.0.1.1


In [2]:
import sympy as sp

import sympy as sp

# Define symbols
ypred, ytrue, union = sp.symbols('ypred ytrue union', real=True, positive=True)

# Define the numerator and denominator
numerator1 = ypred - ytrue
numerator2 = ytrue - ypred

# Define the function using max operation
f = sp.Max(numerator1 / union, numerator2 / union)

# Gradient (partial derivatives w.r.t ypred and ytrue)
grad = [sp.diff(f, var) for var in (ypred, ytrue)]

# Hessian (second-order derivatives)
hess = [[sp.diff(g, var) for var in (ypred, ytrue)] for g in grad]

# Print results
print("Function:")
sp.pprint(f)
print("\nGradient:")
for g in grad:
    sp.pprint(g)
print("\nHessian:")
for row in hess:
    for h in row:
        sp.pprint(h)
        print()


Function:
Max((-ypred + ytrue)/union, (ypred - ytrue)/union)

Gradient:
 ⎛  -ypred + ytrue   ypred - ytrue⎞    ⎛-ypred + ytrue   ypred - ytrue⎞
θ⎜- ────────────── + ─────────────⎟   θ⎜────────────── - ─────────────⎟
 ⎝      union            union    ⎠    ⎝    union            union    ⎠
─────────────────────────────────── - ─────────────────────────────────
               union                                union              
   ⎛  -ypred + ytrue   ypred - ytrue⎞    ⎛-ypred + ytrue   ypred - ytrue⎞
  θ⎜- ────────────── + ─────────────⎟   θ⎜────────────── - ─────────────⎟
   ⎝      union            union    ⎠    ⎝    union            union    ⎠
- ─────────────────────────────────── + ─────────────────────────────────
                 union                                union              

Hessian:
   ⎛  -ypred + ytrue   ypred - ytrue⎞      ⎛-ypred + ytrue   ypred - ytrue⎞
2⋅δ⎜- ────────────── + ─────────────⎟   2⋅δ⎜────────────── - ─────────────⎟
   ⎝      union            union    

In [26]:
import sympy as sp
import numpy as np

def bad_prediction_loss(y, pred):
    """
    Compute the gradient and Hessian for a custom loss function.

    :param y: Ground truth values (array-like).
    :param pred: Predicted values (array-like).
    :return: Gradient and Hessian arrays for the loss.
    """
    # Define symbols for symbolic computation
    ypred, ytrue, union = sp.symbols('ypred ytrue union', real=True, positive=True)

    # Define the function using max operation
    numerator1 = ypred - ytrue
    numerator2 = ytrue - ypred
    

    pred_class = np.argmax(pred, axis=1)

    f = sp.Max(numerator1 / union, numerator2 / union)

    # Compute the gradient (partial derivatives w.r.t ypred and ytrue)
    grad = [sp.diff(f, var) for var in (ypred, ytrue)]

    # Compute the Hessian (second-order derivatives)
    hess = [[sp.diff(g, var) for var in (ypred, ytrue)] for g in grad]

    # Initialize arrays for gradient and Hessian values
    gradient = np.zeros_like(pred)
    hessian = np.zeros((pred.shape[0], pred.shape[1], 2, 2))  # For 2 variables (ypred, ytrue)

    # Loop over each sample
    for i in range(pred.shape[0]):
        target = y[i]
        prediction = pred_class[i]
        union_val = max(prediction, target)  # Calculate union for this sample

        # Substitute the values into gradient and Hessian
        grad_val = [g.subs({ypred: prediction, ytrue: target, union: union_val}).evalf() for g in grad]
        hess_val = [[h.subs({ypred: prediction, ytrue: target, union: union_val}).evalf() for h in h_row] for h_row in hess]
        
        print(grad_val, hess_val)

        # Store the gradient and Hessian for the sample
        gradient[i] = [float(g) for g in grad_val]
        hessian[i] = [[float(h) for h in h_row] for h_row in hess_val]

    # Flatten gradient and Hessian if needed
    gradient = gradient.flatten()
    hessian = hessian.reshape(pred.shape[0], -1)

    return gradient, hessian


In [27]:
# Exemples de données
y = np.array([1])  # Ground truth
pred = np.array([[0.3, 0.5, 0.7]])  # Predictions
print(pred.shape)
# Calcul du gradient et de la hessienne
gradient, hessian = bad_prediction_loss(y, pred)

print("Gradient:", gradient)
print("Hessian:", hessian)


(1, 3)
[0.500000000000000, -0.500000000000000] [[0, 0], [0, 0]]


ValueError: could not broadcast input array from shape (2,) into shape (3,)

In [ ]:
dept = 'departement-69-rhone'
scale = 4
base = 'risk-size-watershed'
path_to_predict = Path('firemen2/firepoint/2x2/train/features_geometry/'+dept)
path_to_gt = Path('firemen/firepoint/2x2/train/features_geometry/'+dept)

image_one = read_object(f'pred_merge.pkl', path_to_predict)
image_two = read_object(f'pred_merge.pkl', path_to_gt)

calculate_iou(image_one, image_two)